In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten, MaxPooling2D, Dense, Conv2D, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
datagen = ImageDataGenerator(rescale=1./255.,
                            horizontal_flip=True,
                            brightness_range=[0.4,1.5],
                            width_shift_range=0.1,
                            height_shift_range=0.1)

In [3]:
train_generator = datagen.flow_from_directory('//DESKTOP-45IP38J/Users/Merve/Desktop/seg_train/seg_train',
                                        batch_size=512,
                                        class_mode='categorical',
                                        target_size=(150,150),
                                        color_mode='rgb')
val_generator = datagen.flow_from_directory('//DESKTOP-45IP38J/Users/Merve/Desktop/seg_test/seg_test/',
                                        batch_size=512,
                                        class_mode='categorical',
                                        target_size=(150,150))

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
model = Sequential([Conv2D(32, input_shape=(150,150,3), kernel_size=(3,3), activation='relu'),
                   MaxPooling2D(pool_size=(2,2)),
                   Dropout(0.25),
                   Conv2D(64, (3,3), activation='relu'),
                   MaxPooling2D(pool_size=(2,2)),
                   Dropout(0.25),
                   Conv2D(128, (3,3), activation='relu'),
                   MaxPooling2D(2,2),
                   Dropout(0.25),
                   Flatten(),
                   Dense(units = 256, activation='relu'),
                   Dropout(0.4),
                   Dense(units = 512, activation='relu')])

model.add(Dense(units = 6, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       7

In [ ]:
history = model.fit(train_generator, batch_size=512,
          epochs=50,
          verbose=1,
          validation_data=val_generator)

Epoch 1/50
28/28 [==============================] - 1285s 45s/step - loss: 2.1801 - accuracy: 0.1965 - val_loss: 1.2884 - val_accuracy: 0.4860
Epoch 2/50
28/28 [==============================] - 1192s 43s/step - loss: 1.2403 - accuracy: 0.5143 - val_loss: 1.1290 - val_accuracy: 0.5623
Epoch 3/50
28/28 [==============================] - 1197s 43s/step - loss: 1.1067 - accuracy: 0.5677 - val_loss: 1.0198 - val_accuracy: 0.6107
Epoch 4/50
28/28 [==============================] - 1215s 43s/step - loss: 1.0113 - accuracy: 0.6075 - val_loss: 0.9422 - val_accuracy: 0.6383
Epoch 5/50
28/28 [==============================] - 1219s 44s/step - loss: 0.9196 - accuracy: 0.6495 - val_loss: 0.8878 - val_accuracy: 0.6880
Epoch 6/50
28/28 [==============================] - 1190s 42s/step - loss: 0.8490 - accuracy: 0.6819 - val_loss: 0.8148 - val_accuracy: 0.7067
Epoch 7/50
28/28 [==============================] - 1200s 43s/step - loss: 0.8134 - accuracy: 0.6998 - val_loss: 0.7638 - val_accuracy: 0.7153

In [ ]:
train_generator.class_indices

In [ ]:
results = model.evaluate(val_generator)
print("test loss, test acc:", results)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['accuracy'], label='Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(borderpad=2)
plt.show()
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['loss'], label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(borderpad=2)
plt.show()

In [ ]:
old_labels = train_generator.class_indices
labels = dict((v,k) for k,v in old_labels.items())
labels

In [ ]:
import cv2

from glob import glob
names = glob('../input/intel-image-classification/seg_pred/seg_pred/*')

plt.figure(figsize=(50,50))

for i in range(87,150):
    plt.subplot(13,5,i-86)
    img = cv2.imread(names[i])
    img = cv2.resize(img,(150,150))
    plt.imshow(img)
    img = np.reshape(img,[1,150,150,3])
    plt.title(labels[model.predict(img).argmax()])
plt.show()